# Visualize Fast-SRGAN Results

In [2]:
from pathlib import Path
import os
from os import listdir
from os.path import isfile, join

from PIL import Image
import matplotlib.pyplot as plt 
   
import math
import numpy as np
import tensorflow as tf

from utils.srgan_tools import resolve_single
from utils.srgan import generator
from utils.niqe import niqe
from utils.tools import load_image, plot_sample
from skimage.metrics import structural_similarity

#import lpips
import torch

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

2022-07-19 16:59:55.088072: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 16:59:55.093314: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-19 16:59:55.093432: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
image_size = (160,120,3)

weights_path = Path('weights/srgan/')

images_path = Path('demo/')
test5_path = Path('demo/set5/Set5/')

fontsize = 32
patch_size = 80

In [5]:
w_list = sorted([f for f in listdir(weights_path) 
           if isfile(join(weights_path, f)) 
           and 'gan' in f
           and 'tiny_prelu' in f])

w_list.insert(0,'pre_generator_tiny_prelu.h5')

with open('weights/bad_weights.txt', 'r') as fp:
    for line in fp:
        w_list.remove(line[:-1])

i_list = [str(images_path.joinpath(f)) for f in listdir(images_path) 
          if isfile(join(images_path, f)) 
          and f.startswith('0')
          and 'sr' not in f]

set5 = [str(test5_path.joinpath(f)) for f in listdir(test5_path) 
        if isfile(join(test5_path, f))
        and 'sr' not in f]

i_list = [*i_list, *set5]

FileNotFoundError: [Errno 2] No such file or directory: 'weights/srgan'

# New heading

In [ ]:
w_list

In [ ]:
i_list

In [ ]:
model = generator(num_filters=32, 
                  num_res_blocks=4, 
                  shape=(image_size), 
                  batch_size=1, 
                  batch_norm=False, 
                  activation='relu',
                  upsampling='TransposeConv')

In [ ]:
#loss_fn_alex = lpips.LPIPS(net='vgg').cuda() # best forward scores

In [ ]:
def multiplot_samples(lr, hr, srs, psnr, niqes, ssims, lps, w_list):

    srs.insert(0,hr)   
    srs.insert(0,lr)   
    
    C = 3
    N = len(srs)
    R = math.ceil(N/C)
    Py = [lr.shape[1]//2 - patch_size//2, lr.shape[1]//2 + patch_size//2]
    Px = [lr.shape[0]//2 - patch_size//2, lr.shape[0]//2 + patch_size//2]
    
    fig, axs = plt.subplots(R, C, figsize=(50,50))
    
    for i, img in enumerate(srs):
        ax = axs.flat[i]
        ax.imshow(img)
        ax.axis('off')
        
        if i is 0:
            ax.set_title('[LR]', fontsize=32)
            ax.set_xlim(Px)
            ax.set_ylim(Py)
            ax.set_ylim(ax.get_ylim()[::-1])
        
        elif i is 1:
            ax.set_title('[HR]', fontsize=32)
            ax.set_xlim([x*4 for x in Px])
            ax.set_ylim([x*4 for x in Py])
            ax.set_ylim(ax.get_ylim()[::-1])
            
        else:
            ax.set_title(
                f'[SR {i-2}]\
                ({w_list[i-2].split("gridsearch_")[-1]})\nPSNR:{psnr[i-2]:.2f} NIQE:{niqes[i-2]:.2f} SSIM:{ssims[i-2]:.2f} LPIPS:{lps[i-2]:.2f}', fontsize=32)
            ax.set_xlim([x*4 for x in Px])
            ax.set_ylim([x*4 for x in Py])
            ax.set_ylim(ax.get_ylim()[::-1])

    fig.show()
    w_list.pop(0)
    #w_list.pop(-1)

In [ ]:
def multipredict_sample(l, w_list):
    
    img = Image.open(l)
    
    hr = np.asarray(img.resize(tuple([i*4 for i in image_size[1::-1]])))
    lr = np.asarray(img.resize(image_size[1::-1]))
    
    srgan = Image.open(l.split('.')[0] + '_sr.png')
    srgan = np.asarray(srgan.resize(tuple([i*4 for i in image_size[1::-1]])))
    
    #lsrgan = Image.open(l.split('.')[0] + '_sr_large.png')
    #lsrgan = np.asarray(srgan.resize(tuple([i*4 for i in image_size[1::-1]])))
        
    srs = []
    psnr = []
    niqes = []
    ssims = []
    lps = []
    
    srs.append(srgan)
    #srs.append(lsrgan)
    psnr, niqes, ssims, lps = evaluate(srs[-1], hr, psnr, niqes, ssims, lps)
    
    for weights in w_list: 
        #print(weights)
        model.load_weights(weights_path.joinpath(weights))    
        srs.append(model.predict(lr[None,...])[0].astype('uint8')[0])
        
        psnr, niqes, ssims, lps = evaluate(srs[-1], hr, psnr, niqes, ssims, lps)
    
    w_list.insert(0,'Original SRGAN') 
    #w_list.insert(-1,'Large SRGAN') 
        
    multiplot_samples(lr, hr, srs, psnr, niqes, ssims, lps, w_list)

In [ ]:
def evaluate(sr, hr, psnr, niqes, ssims, lps):
    
    psnr.append(PSNR(hr, sr))
    niqes.append(niqe(sr))
    ssims.append(structural_similarity(hr, sr, channel_axis=-1, multichannel=True))
    lps.append(0)#float(loss_fn_alex(torch.from_numpy(np.swapaxes(hr,0,-1)).cuda(), 
                  #                torch.from_numpy(np.swapaxes(sr,0,-1)).cuda())))
    
    return psnr, niqes, ssims, lps

In [ ]:
def PSNR(original, compressed, max_pixel=255.0):
    
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0): return 100
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

In [ ]:
for i in i_list:
    multipredict_sample(i, w_list)

In [ ]:
raise

# Remove Bad Weights

In [ ]:
indexes = []

## Remove Bad Weights from List

In [ ]:
with open('weights/bad_weights.txt', 'a') as fp:
    for i in indexes:
        fp.write(f'{w_list[i-1]}\n')

# Generate Standard SRGAN Outputs

In [ ]:
def save_prediction(l, model):
    
    img = Image.open(l)
    shape = tuple(t//4 for t in img.size)
    lr = np.asarray(img.resize(shape))
    print(lr.shape)
    pred = model.predict(lr[None,...]).astype('uint8')[0]
    im = Image.fromarray(pred)
    im.save(l.split('.')[0] + '_sr.png')

In [ ]:
model = generator_old(num_filters=64, num_res_blocks=16)
model.load_weights(weights_path.joinpath('../gan_generator.h5')) 

In [ ]:
for i in i_list:
    print(i)
    # save_prediction(i, model)

# Generate RDN Outputs

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")

In [ ]:
def save_prediction(l, model):
    
    img = Image.open(l)
    shape = tuple(t//4 for t in img.size)
    lr = np.asarray(img.resize(shape))
    lr_img = tf.cast(lr[None], tf.float32)
    pred = model(lr_img).numpy()[0].astype('uint8')
    im = Image.fromarray(pred)
    im.save(l.split('.')[0] + '_sr_rdn.png')

In [ ]:
for i in i_list:
    print(i)
    #save_prediction(i, model)

# Generate Large SRGAN Outputs

In [8]:
def save_prediction(l, model):
    
    img = Image.open(l)
    shape = tuple(t//4 for t in img.size)
    lr = np.asarray(img.resize(shape))
    print(lr.shape)
    pred = model.predict(lr[None,...]).astype('uint8')[0]
    im = Image.fromarray(pred)
    im.save(l.split('.')[0] + '_sr_large.png')

In [7]:
model = generator(scale=4,
                  num_filters=128,
                  num_res_blocks=16,
                  batch_norm=False,
                  activation='relu',
                  upsampling='TransposeConv')

#model.load_weights(weights_path.joinpath('gan_generator_large.h5')) 

In [9]:
for i in i_list:
    print(i)
    save_prediction(i, model)

NameError: name 'i_list' is not defined

In [ ]:
s = 'ciao' \
    'ciao'

In [ ]:
s 

# LPIPS

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

hr = Image.open('demo/0823x4-crop.jpeg')
sr = Image.open('demo/0823x4-crop_sr.png')
srl = Image.open('demo/0823x4-crop_sr_large.png')
srq = Image.open('demo/0823x4-crop_sr_rdn.png')

convert_tensor = transforms.ToTensor()
hr = convert_tensor(hr)
sr = convert_tensor(sr)
srl = convert_tensor(srl)
srq = convert_tensor(srq)

In [ ]:
hr.size(), sr.size(), srl.size(), srq.size()

In [ ]:
float(loss_fn_alex(hr, sr))

# Features from Model

In [10]:
import numpy as np
import tensorflow as tf
from utils.srgan import generator 

In [11]:
m = generator(num_res_blocks=4, num_filters= 32, return_features=False)

TypeError: Cannot iterate over a Tensor with unknown first dimension.

In [ ]:
a = np.random.rand(1,96,96,3)*255
*feats, sr = m(a, training=False)

In [ ]:
len(feats)

In [ ]:
for o in feats:
    print(o.shape)

In [ ]:
feats[0]

In [ ]:
feats[1]

In [ ]:
feats[2]

In [ ]:
feats[3]

In [ ]:
feats[4]

In [ ]:
feats[5]